In [5]:
import pandas as pd
df = pd.read_csv("ValueCo data extract - ENSAE research project.csv")

In [8]:
print(df.columns)

Index(['isin', 'name', 'sector', 'industry', 'country', 'region', 'score_kind',
       'first_quartile', 'consensus', 'third_quartile', 'strength', 'period'],
      dtype='object')


In [9]:
print(df.head())

           isin                        name       sector  \
0  IE0002424939  Dcc public limited company  Industrials   
1  FR0010451203                    Rexel sa  Industrials   
2  FR0010451203                    Rexel sa  Industrials   
3  FR0000120503                 Bouygues sa  Industrials   
4  CH0002432174        Bucher industries ag  Industrials   

                                  industry      country  region score_kind  \
0      Oil and Gas Retail and Distribution      Ireland  Europe        ESG   
1        Electrical Equipment Distributors       France  Europe          S   
2        Electrical Equipment Distributors       France  Europe          E   
3    Integrated Telecommunication Services       France  Europe          G   
4  Construction Machinery and Heavy Trucks  Switzerland  Europe          E   

   first_quartile  consensus  third_quartile  strength      period  
0            52.4       52.4            52.4     100.0  2020-03-01  
1            60.4       60.4    

Pour obtenir tous les scores d'une entreprise particulière (ici Rexel sa):

In [ ]:
def get_company_data(company_name):
    company_df = df[df['name'] == company_name]  
    return company_df

company = "Rexel sa"  
result = get_company_data(company)

In [ ]:
def detect_score_improvement(df, threshold=10):
    results = []
    df['period'] = pd.to_datetime(df['period'])
    df = df.sort_values(by=['name', 'score_kind', 'period'])
    score_columns = ['first_quartile', 'third_quartile', 'consensus']
    for score_type in df['score_kind'].unique():
        subset = df[df['score_kind'] == score_type].copy() 
        for score_col in score_columns:
            subset[f'{score_col}_diff'] = subset.groupby('name')[score_col].diff()
            improved = subset[subset[f'{score_col}_diff'] >= threshold]
            for _, row in improved.iterrows():
                results.append({
                    'Entreprise': row['name'],
                    'Début': subset[(subset['name'] == row['name']) & 
                                    (subset[score_col] == row[score_col] - row[f'{score_col}_diff'])]['period'].min(),
                    'Fin': row['period'],
                    'Type de Score': score_type,
                    'Mesure': score_col  # Indiquer si l'amélioration est sur Q1, Q3 ou consensus
                })
    return pd.DataFrame(results)

improved_companies = detect_score_improvement(df)
print(improved_companies)

                         Entreprise      Début        Fin Type de Score  \
0                   Adecco group ag 2020-01-01 2021-01-01             E   
1                    Aena sme- s.a. 2024-07-01 2024-10-01             E   
2                          Alfen nv 2022-01-01 2022-12-01             E   
3                    Beijer alma ab 2023-01-01 2023-07-01             E   
4                      Bilfinger se 2023-11-01 2024-07-01             E   
..                              ...        ...        ...           ...   
862                       Thales sa 2023-10-01 2023-11-01             S   
863  Ultra Electronics Holdings plc 2023-01-01 2023-07-01             S   
864                    Veidekke asa 2023-08-01 2023-11-01             S   
865                  Webuild s.p.a. 2023-01-01 2023-07-01             S   
866                Zehnder group ag 2023-07-01 2023-11-01             S   

             Mesure  
0    first_quartile  
1    first_quartile  
2    first_quartile  
3    first_